In this notebook I will write the code to collect tweets between January 2019 and June 2022. I will save the tweets to csv.

In [89]:
from pandas import json_normalize
import pip._vendor.requests 
import os
import json
from dotenv import load_dotenv
import csv
import pandas as pd
from datetime import datetime
from operator import itemgetter
import numpy as np
from time import sleep

In [90]:
load_dotenv()

# To get bearer token environment variable
bearer_token = os.environ.get("BEARER-TOKEN")

search_url = "https://api.twitter.com/2/tweets/search/all" 

def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    return r


def connect_to_endpoint(url, params):
    response = pip._vendor.requests.request("GET", search_url, auth=bearer_oauth, params=params)
    # print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [91]:
def getKeywordQueryString():
    keyWordData = pd.read_csv('finalAsianKeywords.csv')
    words = keyWordData['keyword'].tolist()
    keywordQueryString = ""

    for word in words:
        if word != words[-1]:
            keywordQueryString = keywordQueryString + word + " OR "
        else:
            keywordQueryString = keywordQueryString + word
    return keywordQueryString




In [92]:



def query_params(keywordQueryString, startTime, endTime, maxResults, nextToken):

    queryString = f'({keywordQueryString}) lang:en -is:retweet place_country:gb'


    return      {'query': queryString, 
                'start_time': startTime, 
                'end_time': endTime, 
                'max_results': maxResults,
                'tweet.fields': 'text,created_at,id',                
                'next_token': nextToken}
   

In [93]:
def getTweetsToCsv(keywordQueryString, startTime, endTime, maxResults, csvString):
    next_token = {}
    finished = False
    data = []
    mostRecentDate = ""
    count = 0
    while finished == False:
        json_response = connect_to_endpoint(search_url, query_params(keywordQueryString, startTime, endTime, maxResults, next_token))
        for r in json_response['data']:
            date = r['created_at']
            tweetId = r['id']
            tweetText = r['text']
            print(type(tweetText))
            if ("\n" in tweetText):
                tweetText = ''.join(tweetText.splitlines())
            result = [date, tweetId, tweetText]
            data.append(r)
            count +=1
            print(count)
            mostRecentDate = date
        if count > maxResults:
            finished = True
        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
        else:
            finished = True
    
    df = pd.DataFrame(data)
    df.to_csv(csvString, index=False, header=False, mode='a')
    print("most recent date: " + mostRecentDate)
    return mostRecentDate




In [94]:



def twitterScraperLoopUntilComplete(keywordQueryString, startTime, endTime, maxResults, csvString):
    csvFile = open(csvString, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Create headers for the data you want to save, in this example, we only want save these columns in our dataset
    csvWriter.writerow(['date', 'id', 'tweet'])
    csvFile.close()
    end = endTime
    shortFormatStartString = startTime[0:10]
    finished = False
    while finished == False:
        end = getTweetsToCsv(keywordQueryString, startTime, end, maxResults, csvString)
        if (shortFormatStartString == end[0:10]):
            print("YAY IT'S TRUE")
            finished = True
        sleep(4)
    


In [95]:
queryStringOfKeywords = getKeywordQueryString()
startTime = '2020-02-01T00:00:00Z'
endTime = '2020-02-05T00:00:00Z'
# endTime = '2022-06-01T00:00:00Z'
maxResults = 10


twitterScraperLoopUntilComplete(queryStringOfKeywords, startTime, endTime, maxResults, "helloKitty3.csv")

<class 'str'>
1
<class 'str'>
2
<class 'str'>
3
<class 'str'>
4
<class 'str'>
5
<class 'str'>
6
<class 'str'>
7
<class 'str'>
8
<class 'str'>
9


Exception: (429, '{"title":"Too Many Requests","detail":"Too Many Requests","type":"about:blank","status":429}')